In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import os, sys
PATH = os.path.abspath("")[:-3]

In [2]:

# SCRAPPEAR UN USUARIO
from test_user import get_user_data

USER = "ComicGirlAshley"
user_data = get_user_data(USER)
print(user_data)

[245, False, 26733, 99, 20170, 134, 6313, 5.049019607843137, 16.205882352941178, 354.7450980392157, 0.8627450980392157, 0.1568627450980392, 26634]


c:\Users\maria\Documents\Uni\ia orgs\bot_detector\.venv\lib\site-packages\sklearn\feature_extraction\text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [2]:
# Se leen los datos generados con model.py\n",
df_data = pd.read_csv(PATH + "data/training_new.csv").sample(frac=1).reset_index(drop=True)
data = df_data.iloc[:, 1:]
print(data.head())

   factor_rep  verified  followersCount  friendsCount  tweetsCount  \
0         285         1         1854344           416         6640   
1          67         0             435           461         4269   
2         148         0              67            87         1346   
3         554         1            9023          6435         8391   
4         439         0             301           467        27888   

   listedCount  mediaCount  avg_reply  avg_retweet   like_reply  avg_quote  \
0         5867         480  69.900000   293.700000  6037.500000  34.400000   
1            0          10   0.500000     3.630435    15.500000   0.304348   
2            2          15   0.052632     0.026316     0.236842   0.026316   
3          422         428   0.903846     3.923077     6.692308   0.596154   
4            3          33   0.000000     0.000000     0.600000   0.000000   

   hora_rep  followers_diff  clase  
0  0.200000        -1854343  human  
1  0.152174            -435  human  

In [3]:
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
header_names = list(data.columns.values)
for col in data.iloc[:,:-1]:
    data[col] = preprocessing.MinMaxScaler().fit_transform(np.array(data[col]).reshape(-1,1))
x_data = data.iloc[:,:-1].to_numpy()
y_data = data.iloc[:,-1].to_numpy()
x_train, x_test, y_train, y_test = train_test_split(
    x_data, y_data, stratify=y_data, test_size=0.33)

In [4]:
#CONVERTIRLA SALIDA A UN VECTOR DE 0 Y 1
from sklearn.preprocessing import LabelBinarizer

encoder=LabelBinarizer()
label = encoder.fit_transform(y_train)
y_train_transformed = np.hstack((label, 1 - label))
label = encoder.fit_transform(y_test)
y_test_transformed = np.hstack((label, 1 - label))
print(y_train[:5])
print(y_train_transformed[:5])

['human' 'bot' 'human' 'bot' 'bot']
[[1 0]
 [0 1]
 [1 0]
 [0 1]
 [0 1]]


In [5]:
# COMPROBAR TIPO DE LOS DATOS
print(type(x_train))
print(type(y_train_transformed))
# COMPROBAR DIMENSIONES DE LOS DATOS
print(x_train.shape)
print(y_train.shape)
print(y_train_transformed.shape)
print(x_test.shape)
print(y_test.shape)
print(y_test_transformed.shape) 
# OBTENER DIMENSION DE LA ENTRADA Y NÚMERO DE SALIDAS
input_shape = (x_train.shape[1] ,)
num_clases = y_test_transformed.shape[1]

<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
(1616, 13)
(1616,)
(1616, 2)
(796, 13)
(796,)
(796, 2)


# Buscar modelo

In [42]:
# Crear modelo usando sigmoide
from keras.models import Sequential
from keras.layers import Dense
input_shape= (x_train.shape[1] ,)
model = Sequential()
arch = [20, 20, 0, "sigmoid", 0.05]
model.add(Dense(arch[0], input_shape=input_shape, activation=arch[3]))
if arch[1] != 0:
    model.add(Dense(arch[1], input_shape=input_shape, activation=arch[3]))
if arch[2] != 0:
    model.add(Dense(arch[2], input_shape=input_shape, activation=arch[3]))
model.add(Dense(2, activation='softmax'))
model.summary()

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_21 (Dense)            (None, 20)                280       
                                                                 
 dense_22 (Dense)            (None, 20)                420       
                                                                 
 dense_23 (Dense)            (None, 2)                 42        
                                                                 
Total params: 742
Trainable params: 742
Non-trainable params: 0
_________________________________________________________________


In [43]:
# CONFIGURAR MODELO Y ENTRENAMIENTO
from keras.callbacks import EarlyStopping

epochs = 3000
early_stopping = EarlyStopping(monitor='val_loss', patience=1)
model.compile(optimizer=tf.keras.optimizers.SGD(learning_rate=arch[4], momentum=0), metrics=['accuracy','mse'], loss='mean_squared_error')
historico = model.fit(x_train, y_train_transformed, epochs=epochs, verbose=1, validation_split=0.2,
    shuffle=False, validation_freq=1, batch_size=16, callbacks=[early_stopping])

Epoch 1/3000
81/81 [==============================] - 1s 4ms/step - loss: 0.2509 - accuracy: 0.5882 - mse: 0.2509 - val_loss: 0.2480 - val_accuracy: 0.5586 - val_mse: 0.2480
Epoch 2/3000
81/81 [==============================] - 0s 2ms/step - loss: 0.2439 - accuracy: 0.5882 - mse: 0.2439 - val_loss: 0.2478 - val_accuracy: 0.5586 - val_mse: 0.2478
Epoch 3/3000
81/81 [==============================] - 0s 2ms/step - loss: 0.2437 - accuracy: 0.5882 - mse: 0.2437 - val_loss: 0.2476 - val_accuracy: 0.5586 - val_mse: 0.2476
Epoch 4/3000
81/81 [==============================] - 0s 2ms/step - loss: 0.2435 - accuracy: 0.5882 - mse: 0.2435 - val_loss: 0.2474 - val_accuracy: 0.5586 - val_mse: 0.2474
Epoch 5/3000
81/81 [==============================] - 0s 2ms/step - loss: 0.2433 - accuracy: 0.5882 - mse: 0.2433 - val_loss: 0.2473 - val_accuracy: 0.5586 - val_mse: 0.2473
Epoch 6/3000
81/81 [==============================] - 0s 2ms/step - loss: 0.2431 - accuracy: 0.5882 - mse: 0.2431 - val_loss: 0.24

KeyboardInterrupt: 

In [78]:
col_names = ['arquitectura', 'loss', 'val_loss', 'accuracy', 'val_accuracy', 'epoch']
#dff = pd.DataFrame([], columns=col_names)
dff = pd.read_csv(PATH + 'data/output/vals2.csv')
final_epoch=len(historico.history['loss'])-1
print(final_epoch)
tloss = historico.history['loss'][final_epoch-1]
print(tloss)
vloss = historico.history['val_loss'][final_epoch-1]
print(vloss)
tacc = historico.history['accuracy'][final_epoch-1]
print(tacc)
vacc = historico.history['val_accuracy'][final_epoch-1]
print(vacc)
datos = pd.DataFrame([[arch, tloss, vloss, tacc, vacc, final_epoch]], columns=col_names)
datos = pd.concat([dff, datos], ignore_index=True)
datos.to_csv(PATH + 'data/output/vals2.csv', index = False)

11
0.18882940709590912
0.18987923860549927
0.7097523212432861
0.6913580298423767


# Test

In [6]:
from keras.models import Sequential
from keras.layers import Dense
input_shape= (x_train.shape[1] ,)
final_model = Sequential()
arch = [20, 20, 0, "relu", 0.2]
final_model.add(Dense(arch[0], input_shape=input_shape, activation=arch[3]))
final_model.add(Dense(arch[1], input_shape=input_shape, activation=arch[3]))
final_model.add(Dense(2, activation='softmax'))
final_model.summary()

# CONFIGURAR MODELO Y ENTRENAMIENTO
class_weight = {0: 1.5,
                1: 1.}
epochs = 20
final_model.compile(optimizer=tf.keras.optimizers.SGD(learning_rate=arch[4], momentum=0), metrics=['accuracy','mse'], loss='mean_squared_error')
historico = final_model.fit(x_train, y_train_transformed, epochs=epochs, verbose=1, shuffle=False, batch_size=16)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 20)                280       
                                                                 
 dense_1 (Dense)             (None, 20)                420       
                                                                 
 dense_2 (Dense)             (None, 2)                 42        
                                                                 
Total params: 742
Trainable params: 742
Non-trainable params: 0
_________________________________________________________________
Epoch 1/20
101/101 [==============================] - 1s 2ms/step - loss: 0.2328 - accuracy: 0.6343 - mse: 0.2328
Epoch 2/20
101/101 [==============================] - 0s 2ms/step - loss: 0.1999 - accuracy: 0.7079 - mse: 0.1999
Epoch 3/20
101/101 [==============================] - 0s 2ms/step - loss: 0.1941 - accuracy: 0

In [7]:
# EVALUAR MODELO DEFINITIVO
train_results = final_model.evaluate(x_train, y_train_transformed, verbose=1)
test_results= final_model.evaluate(x_test, y_test_transformed, verbose=1)
#EL INDICE 0 ES EL LOSS. EL RESTO, LAS MÉTRICAS QUE SE HAN ESPECIFICADO AL COMPILAR EL MODELO.
# #EN ESTE CASO 'accuracy':1,'mse':2
print(f'Train results-Loss: {train_results[0]} -Accuracy: {train_results[1]} -MSE: {train_results[2]}')
print(f'Test results-Loss: {test_results[0]} -Accuracy: {test_results[1]} -MSE: {test_results[2]}')

25/25 [==============================] - 0s 1ms/step - loss: 0.1975 - accuracy: 0.6595 - mse: 0.1975
Train results-Loss: 0.20223353803157806 -Accuracy: 0.6454207897186279 -MSE: 0.20223353803157806
Test results-Loss: 0.19754737615585327 -Accuracy: 0.659547746181488 -MSE: 0.19754737615585327


In [8]:
# PREDICCIONES DE LAS CLASES
# PREDICCIONES EN BRUTO
raw_testPred= final_model.predict(x_test)
print(raw_testPred[:5])

# PREDICCIONES DE LA CLASE
testPred= np.argmax(raw_testPred, axis=1)
#TRANSFORMAR EL NÚMERO DE COLUMNA EN LA ETIQUETA
class_testPred= encoder.classes_[testPred]
print(class_testPred[:5])

25/25 [==============================] - 0s 1ms/step
[[0.9393924  0.06060758]
 [0.5150908  0.48490918]
 [0.5176003  0.48239973]
 [0.17132911 0.82867086]
 [0.58028096 0.41971898]]
['bot' 'bot' 'bot' 'human' 'bot']


In [9]:
test_targets = y_test
print(class_testPred[:6])
print(test_targets[:6])
comp_df = pd.DataFrame({"predicción": class_testPred, 'probabilidad_bot': raw_testPred.T[0], 'valor real': test_targets}, columns=['predicción', 'probabilidad_bot', 'valor real'])
#guardar en fichero las salidas de test y target
comp_df.to_csv(PATH + 'data/output/salidas_test.csv', index=False)
#mostrar 10 primeras filas
print(comp_df.iloc[:10,:])

['bot' 'bot' 'bot' 'human' 'bot' 'bot']
['human' 'bot' 'bot' 'bot' 'bot' 'human']
  predicción  probabilidad_bot valor real
0        bot          0.939392      human
1        bot          0.515091        bot
2        bot          0.517600        bot
3      human          0.171329        bot
4        bot          0.580281        bot
5        bot          0.556785      human
6        bot          0.573006        bot
7        bot          0.568133        bot
8      human          0.311687        bot
9      human          0.171336        bot


In [10]:
# MATRIZ DE CONFUSIÓN
from sklearn.metrics import confusion_matrix, classification_report
cm=confusion_matrix(y_test,class_testPred)
print('Matriz de confusión')
print(cm)
print('Classification Report')
print(classification_report(y_test,class_testPred))

Matriz de confusión
[[198 266]
 [259  73]]
Classification Report
              precision    recall  f1-score   support

         bot       0.43      0.43      0.43       464
       human       0.22      0.22      0.22       332

    accuracy                           0.34       796
   macro avg       0.32      0.32      0.32       796
weighted avg       0.34      0.34      0.34       796

